In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Lambda
from keras.layers.convolutional import Convolution2D, Cropping2D
import tensorflow as tf
import pickle
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json




Using TensorFlow backend.


In [2]:
column_names = ['center', 'left', 'right', 'steering']
dataframe = pd.read_csv('./data/driving_log.csv')


In [3]:
# Import center images, resize and crop

center_images = []
center = dataframe.center.tolist()
for i in center:
    image = mpimg.imread('./data/' + i)
    image = cv2.resize(image, (200, 66))
    image = image[20:66, :200]
    center_images.append(image)






In [ ]:
# put into a numpy array

%matplotlib inline

center_images = np.asarray(center_images)

plt.imshow(center_images[5])
center_images[5].shape

In [ ]:
# import left images and resize and crop

left_images = []
left = dataframe.left.tolist()

lefty = [i.strip() for i in left]

for i in lefty:
    image = mpimg.imread('./data/' + i)
    image = cv2.resize(image, (200, 66))
    image = image[20:66, :200]
    left_images.append(image)

left_images[0]

In [ ]:
left_images = np.asarray(left_images)

plt.imshow(left_images[5])
left_images[5].shape

In [ ]:
# import right images, resize and crop

right_images = []
right = dataframe.right.tolist()

righty = [i.strip() for i in right]

for i in righty:
    image = mpimg.imread('./data/' + i)
    image = cv2.resize(image, (200, 66))
    image = image[20:66, :200]
    right_images.append(image)

right_images[0]

In [ ]:
right_images = np.asarray(right_images)

plt.imshow(right_images[5])
right_images[5].shape

In [9]:
# add in steering values 

center_values = []
for i in dataframe['steering']:
    center_values.append(i)

center_values = np.asarray(center_values)
len(center_values)
steering = dataframe.steering.tolist()

with open('./allimages.p', 'wb') as f:
    data = pickle.dump(center_images, left_images, right_images, f)
    labels = pickle.dump(center_values, f)


with open('./allimages.p', 'rb') as f:
    data = pickle.load(f)
    labels = pickle.load(f)

In [10]:
# split up data and shuffle

X_train, x_validation, y_train, y_validation = train_test_split(center_images, center_values, test_size = 0.2)

X_train, y_train = shuffle(X_train, y_train)

In [11]:
## Split up steering angle turns for center images

straight, left, right = [], [], []
steer_straight, steer_left, steer_right = [], [], []

index = 0
for i in steering:
    
    if i > 0.15:
        right.append(center_images[index])
        steer_right.append(i)
        index += 1
    elif i < -0.15:
        left.append(center_images[index])
        steer_left.append(i)
        index += 1
    else:
        straight.append(center_images[index])
        steer_straight.append(i)
        index +=1 
    
        

In [12]:
print(len(straight), len(steer_straight))
print(len(right), len(steer_right))
print(len(left), len(steer_left))

6541 6541
902 902
593 593


In [13]:
# adjust steering angle values for left images

for i in range(len(left_images)):
    if steering[i] > 0.15:
        right.append(left_images[i])
        steer_right.append(steering[i] + 0.25)
        
print(len(steer_right), len(right))

1804 1804


In [14]:
# adjust steering angle values for right images

for i in range(len(right_images)):
    if steering[i] < -0.15:
        left.append(right_images[i])
        steer_left.append(steering[i] + -0.25)

print(len(steer_left), len(left))

1186 1186


In [24]:
# convert back to a single numpy array

X_train = np.float32(straight + left + right)
y_train = np.float32(steer_straight + steer_left + steer_right)

In [25]:
epochs = 5
batch_size = 128



In [26]:
kernal = [5, 5]
kernal2 = [3, 3]

model = Sequential()

model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(46, 200, 3)))

model.add(Convolution2D(24, kernal[0], kernal[1], subsample=(2,2), input_shape=(46, 200, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(36, kernal[0], kernal[1], subsample=(2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Convolution2D(48, kernal[0], kernal[1], subsample=(2,2)))
model.add(Activation('relu'))

model.add(Convolution2D(64, kernal2[0], kernal2[1], subsample=(1,1)))
model.add(Activation('relu'))

#model.add(Convolution2D(64, kernal2[0], kernal2[1], subsample=(1,1)))
#model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(1))

#optimizer, loss, accuracy
model.compile(optimizer='adam', loss='mean_squared_error')

#train the model
history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, verbose=2, validation_data=(x_validation, y_validation))

#evaluate the accuracy of the model
model.summary()



Train on 9531 samples, validate on 1608 samples
Epoch 1/5
72s - loss: 0.0224 - val_loss: 0.0085
Epoch 2/5
70s - loss: 0.0121 - val_loss: 0.0076
Epoch 3/5
65s - loss: 0.0114 - val_loss: 0.0074
Epoch 4/5
67s - loss: 0.0105 - val_loss: 0.0070
Epoch 5/5
75s - loss: 0.0099 - val_loss: 0.0068
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 46, 200, 3)    0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 21, 98, 24)    1824        lambda_2[0][0]                   
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 21, 98, 24)    0           convolution2d_5[0][0]            
_____

In [27]:
with open('./model.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)

model.save_weights('model.h5')